Connect to Carla

In [1]:
import carla
import os
import time
import numpy as np
import pygame
import cv2
from datetime import datetime
from PIL import Image


# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
blueprint_library = world.get_blueprint_library()

pygame 2.6.1 (SDL 2.28.4, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


Generate BEV and FOV camera data

In [ ]:
# Destroy existing Prius vehicles
def destroy_existing_priuses():
    actors = world.get_actors().filter('vehicle.toyota.prius')
    for actor in actors:
        actor.destroy()
    print(f"Destroyed {len(actors)} existing Prius vehicles.")

destroy_existing_priuses()

# Spawn Prius at given location
spawn_point = carla.Transform(
    carla.Location(x=-58.468666, y=188.412048, z=7.642653),
    carla.Rotation(yaw=0)
)
prius_bp = blueprint_library.find('vehicle.toyota.prius')
vehicle = world.spawn_actor(prius_bp, spawn_point)
vehicle.set_simulate_physics(True)  # Ensure physics is enabled

# Attach spectator to the vehicle
spectator = world.get_spectator()
spectator.set_transform(vehicle.get_transform())

# Define camera properties
CAMERA_RES = (800, 600)
FOV = 90
SAVE_PATH = "carla_dataset"
os.makedirs(SAVE_PATH, exist_ok=True)

# Attach front-facing camera
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', str(CAMERA_RES[0]))
camera_bp.set_attribute('image_size_y', str(CAMERA_RES[1]))
camera_bp.set_attribute('fov', str(FOV))

fov_camera_transform = carla.Transform(carla.Location(x=2, z=1.5))  # Front-mounted
bev_camera_transform = carla.Transform(carla.Location(z=15), carla.Rotation(pitch=-90))  # BEV

fov_camera = world.spawn_actor(camera_bp, fov_camera_transform, attach_to=vehicle)
bev_camera = world.spawn_actor(camera_bp, bev_camera_transform, attach_to=vehicle)

# Function to save images
def save_image(image, prefix):
    image_data = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_data = image_data.reshape((image.height, image.width, 4))[:, :, :3]  # Remove alpha channel
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")
    filename = os.path.join(SAVE_PATH, f"{prefix}_{timestamp}.png")
    from PIL import Image
    Image.fromarray(image_data).save(filename)
    print(f"Saved: {filename}")

# Initialize pygame for keyboard control
pygame.init()
screen = pygame.display.set_mode((400, 300))
clock = pygame.time.Clock()
control = carla.VehicleControl()

# Listen for camera images continuously
fov_camera.listen(lambda image: save_image(image, "fov"))
bev_camera.listen(lambda image: save_image(image, "bev"))

running = True
while running:
    pygame.event.pump()  # Process pygame events
    keys = pygame.key.get_pressed()
    
    control.throttle = 1.0 if keys[pygame.K_w] else 0.0
    control.reverse = True if keys[pygame.K_s] else False
    control.brake = 1.0 if keys[pygame.K_s] and not control.reverse else 0.0
    control.steer = -0.5 if keys[pygame.K_a] else (0.5 if keys[pygame.K_d] else 0.0)
    
    vehicle.apply_control(control)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    clock.tick(30)

# Cleanup
fov_camera.destroy()
bev_camera.destroy()
vehicle.destroy()
destroy_existing_priuses()
pygame.quit()
print("Dataset collection complete!")


Generate LIDAR and FOV Camera Data; Overlay Code

In [2]:
from PIL import Image

In [10]:
# Destroy existing Prius vehicles
def destroy_existing_priuses():
    actors = world.get_actors().filter('vehicle.toyota.prius')
    for actor in actors:
        actor.destroy()
    print(f"Destroyed {len(actors)} existing Prius vehicles.")

destroy_existing_priuses()

# Spawn Prius at given location
spawn_point = carla.Transform(
    carla.Location(x=-58.468666, y=188.412048, z=7.642653),
    carla.Rotation(yaw=0)
)
prius_bp = blueprint_library.find('vehicle.toyota.prius')
vehicle = world.spawn_actor(prius_bp, spawn_point)
vehicle.set_simulate_physics(True)  # Ensure physics is enabled

# # Attach spectator to the vehicle
# spectator = world.get_spectator()
# spectator.set_transform(vehicle.get_transform())

# Define camera and LiDAR properties
CAMERA_RES = (800, 600)
FOV = 90
SAVE_PATH = "fused_camera_lidar"
os.makedirs(SAVE_PATH, exist_ok=True)

# Attach front-facing camera
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', str(CAMERA_RES[0]))
camera_bp.set_attribute('image_size_y', str(CAMERA_RES[1]))
camera_bp.set_attribute('fov', str(FOV))

fov_camera_transform = carla.Transform(carla.Location(x=2, z=1.5))  # Front-mounted

fov_camera = world.spawn_actor(camera_bp, fov_camera_transform, attach_to=vehicle)

# Attach LiDAR sensor
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
# lidar_bp.set_attribute('points_per_second', '50000')
lidar_bp.set_attribute('range', '50')  # Maximum range for LiDAR (in meters)

lidar_transform = carla.Transform(carla.Location(x=0, z=2))  # LiDAR mounted on vehicle roof
lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)

# Function to save images and LiDAR data
def save_data(types, image=None, lidar_data=None):
    print("inside save data")
    lidar_points = np.frombuffer(lidar_data.raw_data, dtype=np.float32)
    lidar_points = lidar_points.reshape([-1, 4])  # LiDAR data format: [x, y, z, intensity]
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")
    image_data = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_data = image_data.reshape((image.height, image.width, 4))[:, :, :3]  # Remove alpha channel
    if "camera" in types:
        # Save Camera Image (FOV)
        filename = os.path.join(SAVE_PATH, f"fov_{timestamp}.png")
        Image.fromarray(image_data).save(filename)
        print(f"Saved camera image: {filename}")

    if "lidar" in types:
        # Save LiDAR Data (as NumPy .npy file)
        lidar_filename = os.path.join(SAVE_PATH, f"lidar_{timestamp}.npy")
        np.save(lidar_filename, lidar_points)
        print(f"Saved LiDAR data: {lidar_filename}")

    if "overlaid" in types:
        # Function to apply a heatmap to visualize depth
        def apply_heatmap(depth_map):
            normed_depth = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
            heatmap = cv2.applyColorMap(normed_depth.astype(np.uint8), cv2.COLORMAP_JET)  # Applying heatmap color map
            return heatmap

        lidar_points = lidar_data[:, :3]  # Use just x, y, z for the overlay
        # Create a depth map as a NumPy array (same resolution as the camera image)
        depth_map = np.zeros((CAMERA_RES[1], CAMERA_RES[0]), dtype=np.float32)
        # Loop through LiDAR points and project them onto the image
        for point in lidar_points:
            x, y, z = point
            if z > 0:  # Ensure that LiDAR points are in front of the camera
                # Simple projection (can improve with calibration matrix)
                screen_x = int(CAMERA_RES[0] // 2 + x / z * 200)  # Scale factor for projection
                screen_y = int(CAMERA_RES[1] // 2 - y / z * 200)
                
                if 0 <= screen_x < CAMERA_RES[0] and 0 <= screen_y < CAMERA_RES[1]:
                    # Store the closest depth value for each pixel (in case of multiple LiDAR points in the same pixel)
                    depth_map[screen_y, screen_x] = min(depth_map[screen_y, screen_x], z)
                    
                    # You can visualize the LiDAR points directly in the image (red color)
                    image_data[screen_y, screen_x] = [255, 0, 0]  # Red color for LiDAR points
        
        # Save the original image (with LiDAR points)
        image_filename = os.path.join(SAVE_PATH, f"fov/image_{timestamp}.png")
        Image.fromarray(image_data).save(image_filename)
        print(f"Saved: {image_filename}")

        # Apply the heatmap overlay to visualize depth
        heatmap_image = apply_heatmap(depth_map)

        # Save the heatmap overlay image
        heatmap_filename = os.path.join(SAVE_PATH, f"heatmap/heatmap_{timestamp}.png")
        Image.fromarray(heatmap_image).save(heatmap_filename)
        print(f"Saved: {heatmap_filename}")

        # Save the depth map as a NumPy array (you can load this array later)
        depth_filename = os.path.join(SAVE_PATH, f"depth_map/depth_map_{timestamp}.npy")
        np.save(depth_filename, depth_map)
        print(f"Saved: {depth_filename}")

                

# Initialize pygame for keyboard control
pygame.init()
screen = pygame.display.set_mode((400, 300))
clock = pygame.time.Clock()
control = carla.VehicleControl()

# Listen for camera and LiDAR data
fov_image = None
def set_fov_image(image):
    global fov_image
    fov_image = image
fov_camera.listen(lambda image: set_fov_image(image))
lidar.listen(lambda lidar_data: save_data(["overlaid"], fov_image, lidar_data))

running = True
while running:
    pygame.event.pump()  # Process pygame events
    keys = pygame.key.get_pressed()
    
    control.throttle = 1.0 if keys[pygame.K_w] else 0.0
    control.reverse = True if keys[pygame.K_s] else False
    control.brake = 1.0 if keys[pygame.K_s] and not control.reverse else 0.0
    control.steer = -0.5 if keys[pygame.K_a] else (0.5 if keys[pygame.K_d] else 0.0)
    
    vehicle.apply_control(control)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    clock.tick(30)

# Cleanup
fov_camera.destroy()
lidar.destroy()
vehicle.destroy()
destroy_existing_priuses()
pygame.quit()
print("Dataset collection complete!")


In [12]:
import carla
import pygame
import numpy as np
import cv2

# Initialize pygame for manual control
pygame.init()
screen = pygame.display.set_mode((400, 300))
pygame.display.set_caption("CARLA Manual Control")

# Enable synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True  # Sync mode ON
settings.fixed_delta_seconds = 0.05  # 20 FPS
world.apply_settings(settings)

# Get blueprints
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter("vehicle.toyota.prius")[0]
rgb_bp = blueprint_library.find("sensor.camera.rgb")
depth_bp = blueprint_library.find("sensor.camera.depth")

# Set camera attributes
rgb_bp.set_attribute("image_size_x", "800")
rgb_bp.set_attribute("image_size_y", "600")
rgb_bp.set_attribute("fov", "90")
depth_bp.set_attribute("image_size_x", "800")
depth_bp.set_attribute("image_size_y", "600")
depth_bp.set_attribute("fov", "90")

# Spawn vehicle
spawn_point = carla.Transform(carla.Location(x=-58.468666, y=188.412048, z=7.642653), carla.Rotation(yaw=0))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

# Attach cameras to the vehicle
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.0))
rgb_camera = world.spawn_actor(rgb_bp, camera_transform, attach_to=vehicle)
depth_camera = world.spawn_actor(depth_bp, camera_transform, attach_to=vehicle)

# Global variables to store images
rgb_image = None
depth_image = None

def process_rgb(image):
    global rgb_image
    rgb_image = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))[:, :, :3]

def process_depth(image):
    global depth_image
    depth_array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    depth_image = (depth_array[:, :, 0] + depth_array[:, :, 1] * 256 + depth_array[:, :, 2] * 256 * 256) / (256 * 256 - 1) * 1000

# Start listening
rgb_camera.listen(lambda image: process_rgb(image))
depth_camera.listen(lambda image: process_depth(image))

# Manual control setup
control = carla.VehicleControl()
frame_count = 0
reverse_mode = False  # Track reverse state

try:
    running = True
    while running:
        world.tick()  # Wait for sensors to update

        # Get keyboard input
        keys = pygame.key.get_pressed()

        # Toggle reverse mode
        if keys[pygame.K_s]:  
            reverse_mode = not reverse_mode  
        
        # Set reverse state
        control.reverse = reverse_mode  

        if keys[pygame.K_UP]:  # Accelerate
            control.throttle = min(control.throttle + 0.05, 1.0)
        else:
            control.throttle = 0.0

        if keys[pygame.K_DOWN]:  # Brake
            control.brake = min(control.brake + 0.2, 1.0)
        else:
            control.brake = 0.0

        if keys[pygame.K_LEFT]:  # Steer Left
            control.steer = max(control.steer - 0.05, -1.0)
        elif keys[pygame.K_RIGHT]:  # Steer Right
            control.steer = min(control.steer + 0.05, 1.0)
        else:
            control.steer = 0.0  # Reset steering when no key is pressed

        vehicle.apply_control(control)

        # Quit if Q is pressed
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_q):
                running = False

        # Visualization & Saving
        if rgb_image is not None and depth_image is not None:
            depth_colored = cv2.applyColorMap((depth_image / 1000.0 * 255).astype(np.uint8), cv2.COLORMAP_JET)
            # combined = np.hstack((rgb_image, depth_colored))
            # cv2.imshow("RGB & Depth", combined)

            # Save images every 50 frames
            if frame_count % 50 == 0:
                np.save(f"depth_frame_{frame_count}.npy", depth_image)
                cv2.imwrite(f"rgb_frame_{frame_count}.png", rgb_image)
                cv2.imwrite(f"depth_frame_{frame_count}.png", depth_colored)

            frame_count += 1
            cv2.waitKey(1)

finally:
    # Cleanup
    rgb_camera.stop()
    depth_camera.stop()
    rgb_camera.destroy()
    depth_camera.destroy()
    vehicle.destroy()
    world.apply_settings(settings)
    pygame.quit()
    cv2.destroyAllWindows()